# Data Cleaning for Visualization Dashboard Project

## Setup

### Import Libraries

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import geojson
import json
import os

### Set file locations

In [2]:
# input file containing dataset as acquired
superfund_sites_file = "../01_Data_Acquisition/priorities_list_full.json"

# csv file for inspecting the results of the initial data import
import_inspection_file = "verification_files/priorities_list_full.csv"

# csv file for inspecting the results of data cleaning
cleaning_inspection_file = "verification_files/priorities_list_full_cleaned.csv"

# csv file for inspecting data prepped for visualization
viz_inspection_file = "verification_files/superfund_site_data.csv"

# geojson output file for visualizations
geojson_output_file = "../03_Dataset/superfund_site_geojson.js"

## Exploring the Dataset

### Bring in the JSON Dataset

In [3]:
superfund_sites_df = pd.read_json(superfund_sites_file)

### Create a list of column names
We copy this list over to a spreadsheet to compare it with simialr lists from the census data files. Notably, the JSON file includes both census block and tract IDs, along with the full FIPS code. It is especially convenient that the JSON file includes data of particular interest, including the latidtude and longitude of each site, the site ID, the site 'score', and socioeconomic values for the population within the same census boundaries.

In [4]:
superfund_column_names = list(superfund_sites_df.columns.values)
superfund_column_names

['FIPS_Full',
 'address',
 'city',
 'date_added',
 'federal_facility_ind',
 'federal_register_url',
 'geocode_source',
 'latitude',
 'longitude',
 'site_epa_id',
 'site_name',
 'site_narrative_url',
 'site_progress_url',
 'site_score',
 'text',
 'FIPS_Block_Group',
 'State',
 'State_name',
 'County',
 'County_name',
 'Tract',
 'Block_Group',
 'Flag',
 'LAND_AREA',
 'AIAN_LAND',
 'URBANIZED_AREA_POP_CEN_2010',
 'URBAN_CLUSTER_POP_CEN_2010',
 'RURAL_POP_CEN_2010',
 'Tot_Population_CEN_2010',
 'Tot_Population_ACS_09_13',
 'Tot_Population_ACSMOE_09_13',
 'Males_CEN_2010',
 'Males_ACS_09_13',
 'Males_ACSMOE_09_13',
 'Females_CEN_2010',
 'Females_ACS_09_13',
 'Females_ACSMOE_09_13',
 'Pop_under_5_CEN_2010',
 'Pop_under_5_ACS_09_13',
 'Pop_under_5_ACSMOE_09_13',
 'Pop_5_17_CEN_2010',
 'Pop_5_17_ACS_09_13',
 'Pop_5_17_ACSMOE_09_13',
 'Pop_18_24_CEN_2010',
 'Pop_18_24_ACS_09_13',
 'Pop_18_24_ACSMOE_09_13',
 'Pop_25_44_CEN_2010',
 'Pop_25_44_ACS_09_13',
 'Pop_25_44_ACSMOE_09_13',
 'Pop_45_64_CEN

### Create a list of column types
All the JSON data came in as either integers, strings, or floats.

In [5]:
superfund_column_types = list(superfund_sites_df.dtypes)
superfund_column_types

[dtype('int64'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('O'),
 dtype('int64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),

### Take a peek at the head

In [6]:
superfund_sites_df.head()

,FIPS_Full,address,city,date_added,federal_facility_ind,federal_register_url,geocode_source,latitude,longitude,site_epa_id,...,pct_TEA_MailOutMailBack_CEN_2010,pct_TEA_Update_Leave_CEN_2010,pct_Census_Mail_Returns_CEN_2010,pct_Vacant_CEN_2010,pct_Deletes_CEN_2010,pct_Census_UAA_CEN_2010,pct_Mailback_Count_CEN_2010,pct_FRST_FRMS_CEN_2010,pct_RPLCMNT_FRMS_CEN_2010,pct_BILQ_Mailout_count_CEN_2010
0,11210113001044,"STATE HWY 235, CHILDERSBURG, AL 35044",Childersburg,07/22/1987,Yes,https://semspub.epa.gov/src/document/11/189629,full_address,33.286432,-86.358807,AL6210020008,...,100.00,0.00,74.03,3.58,0.0,10.90,85.52,67.16,6.87,0.00
1,11170301021051,"570 HIGHWAY 231 NORTH, VINCENT, AL 35178",Vincent,09/18/2012,No,http://www.gpo.gov/fdsys/pkg/FR-2012-09-18/pdf...,full_address,33.399231,-86.404985,ALD004022448,...,100.00,0.00,55.28,2.44,0.0,31.06,66.50,48.62,6.67,0.00
2,11010060001057,"3017 BIRMINGHAM HWY, MONTGOMERY, AL 36108",Montgomery,08/30/1990,No,https://semspub.epa.gov/src/document/11/189637,full_address,32.367036,-86.372935,ALD007454085,...,100.00,0.00,27.33,2.84,0.0,54.19,42.97,23.06,4.27,0.00
3,60375021002013,"12504 AND 12512 E. WHITTIER BOULEVARD, WHITTIE...",Whittier,01/19/1999,No,http://www.gpo.gov/fdsys/pkg/FR-1999-01-19/pdf...,full_address,33.969447,-118.043443,CAD042245001,...,100.00,0.00,72.02,1.46,0.0,3.58,94.96,72.02,0.00,97.88
4,420659503005008,"ROUTE 950, FALLS CREEK, PA 15840",Falls Creek,09/14/2005,No,http://www.gpo.gov/fdsys/pkg/FR-2005-09-14/pdf...,full_address,41.145061,-78.804365,PAD001222025,...,46.31,53.69,73.60,4.03,4.7,3.80,87.47,71.59,2.01,0.00


### Export the dataframe as a csv for manual inspection

In [14]:
superfund_sites_df.to_csv(import_inspection_file, index = False)

## Cleaning the Data

### Define the desired columns
The columns listed below were selected for potential visualization.

In [7]:
# Create a filtered dataframe from specific columns
desired_columns = ["address",
                "city",
                "latitude",
                "longitude",
                "site_score",
                "text",
                "State_name",
                "County_name",
                "Tract",
                "Block_Group",
                "Tot_Population_CEN_2010",
                "Hispanic_CEN_2010",
                "NH_Blk_alone_CEN_2010",
                "NH_AIAN_alone_CEN_2010",
                "NH_Asian_alone_CEN_2010",
                "NH_NHOPI_alone_CEN_2010",
                "NH_SOR_alone_CEN_2010",
                "College_ACS_09_13",
                "No_Health_Ins_ACS_09_13",
                "Med_HHD_Inc_BG_ACS_09_13",
                "Aggregate_HH_INC_ACS_09_13",
                "Tot_Vacant_Units_CEN_2010",
                "Renter_Occp_HU_CEN_2010",
                "Owner_Occp_HU_CEN_2010",
                "No_Plumb_ACS_09_13",
                "Med_House_Value_BG_ACS_09_13",
                "pct_Hispanic_CEN_2010",
                "pct_NH_Blk_alone_CEN_2010",
                "pct_NH_AIAN_alone_CEN_2010",
                "pct_NH_Asian_alone_CEN_2010",
                "pct_NH_NHOPI_alone_CEN_2010",
                "pct_NH_SOR_alone_CEN_2010",
                "pct_Not_HS_Grad_ACS_09_13",
                "pct_No_Health_Ins_ACS_09_13",
                "pct_Vacant_Units_CEN_2010",
                "pct_Renter_Occp_HU_CEN_2010",
                "pct_Owner_Occp_HU_CEN_2010",
                "pct_No_Plumb_ACS_09_13"]

### Create the filtered dataframe

In [17]:
# pull in the desired columns
superfund_sites_cleaned = superfund_sites_df[desired_columns].copy()

# convert the column names to lowercase to avoid problems with postgreSQL.
superfund_sites_cleaned.columns= superfund_sites_cleaned.columns.str.lower()

# make sure the census block groups and tracts come through as integers
superfund_sites_cleaned.tract = superfund_sites_cleaned.tract.fillna(0.0).astype(int)
superfund_sites_cleaned.block_group = superfund_sites_cleaned.block_group.fillna(0.0).astype(int)

# convert currency strings to floats
superfund_sites_cleaned[['med_hhd_inc_bg_acs_09_13']] = (superfund_sites_cleaned['med_hhd_inc_bg_acs_09_13'].replace( '[\$,)]','', regex=True )
                     .replace( '[(]','-',   regex=True ).astype(float))

superfund_sites_cleaned[['aggregate_hh_inc_acs_09_13']] = (superfund_sites_cleaned['aggregate_hh_inc_acs_09_13'].replace( '[\$,)]','', regex=True )
                     .replace( '[(]','-',   regex=True ).astype(float))

superfund_sites_cleaned[['med_house_value_bg_acs_09_13']] = (superfund_sites_cleaned['med_house_value_bg_acs_09_13'].replace( '[\$,)]','', regex=True )
                     .replace( '[(]','-',   regex=True ).astype(float))

# rename the 'text' column to avoid potential confusion
superfund_sites_cleaned.rename(columns={"text": "site_text"}, inplace = True)

# fill in nulls with zeros
superfund_sites_cleaned.fillna(0, inplace = True)

# check the head
superfund_sites_cleaned.head()

,address,city,latitude,longitude,site_score,site_text,state_name,county_name,tract,block_group,...,pct_nh_aian_alone_cen_2010,pct_nh_asian_alone_cen_2010,pct_nh_nhopi_alone_cen_2010,pct_nh_sor_alone_cen_2010,pct_not_hs_grad_acs_09_13,pct_no_health_ins_acs_09_13,pct_vacant_units_cen_2010,pct_renter_occp_hu_cen_2010,pct_owner_occp_hu_cen_2010,pct_no_plumb_acs_09_13
0,"STATE HWY 235, CHILDERSBURG, AL 35044",Childersburg,33.286432,-86.358807,36.83,NPL SITE LISTING NARRATIVE\n\n\nNPL Site Narra...,Alabama,Talladega County,11300,1,...,0.07,0.21,0.00,0.14,17.62,5.32,12.89,19.02,80.98,0.00
1,"570 HIGHWAY 231 NORTH, VINCENT, AL 35178",Vincent,33.399231,-86.404985,30.20,NATIONAL PRIORITIES LIST (NPL) SITE LISTING NA...,Alabama,Shelby County,30102,1,...,0.29,0.00,0.00,0.00,30.23,14.50,9.02,16.00,84.00,7.02
2,"3017 BIRMINGHAM HWY, MONTGOMERY, AL 36108",Montgomery,32.367036,-86.372935,44.46,NPL SITE LISTING NARRATIVE\n\n\nNPL Site Narra...,Alabama,Montgomery County,6000,1,...,0.00,0.00,0.00,0.11,50.87,33.92,24.11,33.11,66.89,0.00
3,"12504 AND 12512 E. WHITTIER BOULEVARD, WHITTIE...",Whittier,33.969447,-118.043443,30.94,NPL SITE LISTING NARRATIVE\n\n\nNPL Site Narra...,California,Los Angeles County,502100,2,...,0.42,1.69,0.04,0.08,31.43,22.99,3.60,61.83,38.17,3.52
4,"ROUTE 950, FALLS CREEK, PA 15840",Falls Creek,41.145061,-78.804365,30.22,NATIONAL PRIORITIES LIST (NPL) SITE LISTING NA...,Pennsylvania,Jefferson County,950300,5,...,0.20,0.20,0.00,0.00,7.70,10.70,7.53,27.91,72.09,2.06


### Create a list of column types
A list of column types is useful for checking against the database table. (We need to verify that there are no surprises, like currency stored as a string.)

In [9]:
superfund_cleaned_column_types = list(superfund_sites_cleaned.dtypes)
dtype_reference = list(zip(superfund_sites_cleaned.columns.values, superfund_cleaned_column_types))
dtype_reference

[('address', dtype('O')),
 ('city', dtype('O')),
 ('latitude', dtype('float64')),
 ('longitude', dtype('float64')),
 ('site_score', dtype('float64')),
 ('site_text', dtype('O')),
 ('state_name', dtype('O')),
 ('county_name', dtype('O')),
 ('tract', dtype('int32')),
 ('block_group', dtype('int32')),
 ('tot_population_cen_2010', dtype('float64')),
 ('hispanic_cen_2010', dtype('float64')),
 ('nh_blk_alone_cen_2010', dtype('float64')),
 ('nh_aian_alone_cen_2010', dtype('float64')),
 ('nh_asian_alone_cen_2010', dtype('float64')),
 ('nh_nhopi_alone_cen_2010', dtype('float64')),
 ('nh_sor_alone_cen_2010', dtype('float64')),
 ('college_acs_09_13', dtype('float64')),
 ('no_health_ins_acs_09_13', dtype('float64')),
 ('med_hhd_inc_bg_acs_09_13', dtype('float64')),
 ('aggregate_hh_inc_acs_09_13', dtype('float64')),
 ('tot_vacant_units_cen_2010', dtype('float64')),
 ('renter_occp_hu_cen_2010', dtype('float64')),
 ('owner_occp_hu_cen_2010', dtype('float64')),
 ('no_plumb_acs_09_13', dtype('float64')

### Export the cleaned dataframe to csv for inspection

In [18]:
superfund_sites_cleaned.to_csv(cleaning_inspection_file, index = False)

## Load the Cleaned Data into Pre-Analysis Database
The cleaned data is loaded into a database for storage. Dat pulled from here will be used for analysis.

### Connect to the database

In [19]:
rds_connection_string = "postgres:postgres@localhost:5432/project_2_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [20]:
# Confirm tables
engine.table_names()

['superfund_data_table', 'superfund_analysis_table']

### Load the cleaned dataframe into the database

In [13]:
superfund_sites_cleaned.to_sql(name='superfund_data_table', con=engine, if_exists='append', index=False)

### Confirm the Load

In [21]:
pd.read_sql_query('select * from superfund_data_table', con=engine).head()

,id,address,city,latitude,longitude,site_score,site_text,state_name,county_name,tract,...,pct_nh_aian_alone_cen_2010,pct_nh_asian_alone_cen_2010,pct_nh_nhopi_alone_cen_2010,pct_nh_sor_alone_cen_2010,pct_not_hs_grad_acs_09_13,pct_no_health_ins_acs_09_13,pct_vacant_units_cen_2010,pct_renter_occp_hu_cen_2010,pct_owner_occp_hu_cen_2010,pct_no_plumb_acs_09_13
0,1,"STATE HWY 235, CHILDERSBURG, AL 35044",Childersburg,33.286432,-86.358807,36.83,NPL SITE LISTING NARRATIVE\n\n\nNPL Site Narra...,Alabama,Talladega County,11300,...,0.07,0.21,0.00,0.14,17.62,5.32,12.89,19.02,80.98,0.00
1,2,"570 HIGHWAY 231 NORTH, VINCENT, AL 35178",Vincent,33.399231,-86.404985,30.20,NATIONAL PRIORITIES LIST (NPL) SITE LISTING NA...,Alabama,Shelby County,30102,...,0.29,0.00,0.00,0.00,30.23,14.50,9.02,16.00,84.00,7.02
2,3,"3017 BIRMINGHAM HWY, MONTGOMERY, AL 36108",Montgomery,32.367036,-86.372935,44.46,NPL SITE LISTING NARRATIVE\n\n\nNPL Site Narra...,Alabama,Montgomery County,6000,...,0.00,0.00,0.00,0.11,50.87,33.92,24.11,33.11,66.89,0.00
3,4,"12504 AND 12512 E. WHITTIER BOULEVARD, WHITTIE...",Whittier,33.969447,-118.043443,30.94,NPL SITE LISTING NARRATIVE\n\n\nNPL Site Narra...,California,Los Angeles County,502100,...,0.42,1.69,0.04,0.08,31.43,22.99,3.60,61.83,38.17,3.52
4,5,"ROUTE 950, FALLS CREEK, PA 15840",Falls Creek,41.145061,-78.804365,30.22,NATIONAL PRIORITIES LIST (NPL) SITE LISTING NA...,Pennsylvania,Jefferson County,950300,...,0.20,0.20,0.00,0.00,7.70,10.70,7.53,27.91,72.09,2.06


## Analysis

In [22]:
# Grab the data from the database
superfund_sites_for_analysis = pd.read_sql_query('select * from superfund_data_table', con=engine)

In [23]:
superfund_sites_analyzed = superfund_sites_for_analysis.copy()

In [24]:
superfund_sites_analyzed["pct_poc"] = (
    superfund_sites_analyzed.pct_hispanic_cen_2010 +\
    superfund_sites_analyzed.pct_nh_blk_alone_cen_2010 +\
    superfund_sites_analyzed.pct_nh_aian_alone_cen_2010 +\
    superfund_sites_analyzed.pct_nh_asian_alone_cen_2010 +\
    superfund_sites_analyzed.pct_nh_nhopi_alone_cen_2010 +\
    superfund_sites_analyzed.pct_nh_sor_alone_cen_2010)

In [25]:
superfund_sites_analyzed.head()

,id,address,city,latitude,longitude,site_score,site_text,state_name,county_name,tract,...,pct_nh_asian_alone_cen_2010,pct_nh_nhopi_alone_cen_2010,pct_nh_sor_alone_cen_2010,pct_not_hs_grad_acs_09_13,pct_no_health_ins_acs_09_13,pct_vacant_units_cen_2010,pct_renter_occp_hu_cen_2010,pct_owner_occp_hu_cen_2010,pct_no_plumb_acs_09_13,pct_poc
0,1,"STATE HWY 235, CHILDERSBURG, AL 35044",Childersburg,33.286432,-86.358807,36.83,NPL SITE LISTING NARRATIVE\n\n\nNPL Site Narra...,Alabama,Talladega County,11300,...,0.21,0.00,0.14,17.62,5.32,12.89,19.02,80.98,0.00,20.29
1,2,"570 HIGHWAY 231 NORTH, VINCENT, AL 35178",Vincent,33.399231,-86.404985,30.20,NATIONAL PRIORITIES LIST (NPL) SITE LISTING NA...,Alabama,Shelby County,30102,...,0.00,0.00,0.00,30.23,14.50,9.02,16.00,84.00,7.02,30.49
2,3,"3017 BIRMINGHAM HWY, MONTGOMERY, AL 36108",Montgomery,32.367036,-86.372935,44.46,NPL SITE LISTING NARRATIVE\n\n\nNPL Site Narra...,Alabama,Montgomery County,6000,...,0.00,0.00,0.11,50.87,33.92,24.11,33.11,66.89,0.00,81.57
3,4,"12504 AND 12512 E. WHITTIER BOULEVARD, WHITTIE...",Whittier,33.969447,-118.043443,30.94,NPL SITE LISTING NARRATIVE\n\n\nNPL Site Narra...,California,Los Angeles County,502100,...,1.69,0.04,0.08,31.43,22.99,3.60,61.83,38.17,3.52,85.29
4,5,"ROUTE 950, FALLS CREEK, PA 15840",Falls Creek,41.145061,-78.804365,30.22,NATIONAL PRIORITIES LIST (NPL) SITE LISTING NA...,Pennsylvania,Jefferson County,950300,...,0.20,0.00,0.00,7.70,10.70,7.53,27.91,72.09,2.06,1.31


## Load the Analyzed Data into Post-Analysis Database
The cleaned data is loaded into a database for storage. Dat pulled from here will be used for analysis.

In [9]:
rds_connection_string = "postgres:postgres@localhost:5432/project_2_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [10]:
# Confirm tables
engine.table_names()

['superfund_data_table', 'superfund_analysis_table']

In [21]:
superfund_sites_analyzed.to_sql(name='superfund_analysis_table', con=engine, if_exists='append', index=False)

### Confirm the Load

In [8]:
pd.read_sql_query('select * from superfund_analysis_table', con=engine).head()

,id,address,city,latitude,longitude,site_score,site_text,state_name,county_name,tract,...,pct_nh_asian_alone_cen_2010,pct_nh_nhopi_alone_cen_2010,pct_nh_sor_alone_cen_2010,pct_not_hs_grad_acs_09_13,pct_no_health_ins_acs_09_13,pct_vacant_units_cen_2010,pct_renter_occp_hu_cen_2010,pct_owner_occp_hu_cen_2010,pct_no_plumb_acs_09_13,pct_poc
0,1,"STATE HWY 235, CHILDERSBURG, AL 35044",Childersburg,33.286432,-86.358807,36.83,NPL SITE LISTING NARRATIVE\n\n\nNPL Site Narra...,Alabama,Talladega County,11300,...,0.21,0.00,0.14,17.62,5.32,12.89,19.02,80.98,0.00,20.29
1,2,"570 HIGHWAY 231 NORTH, VINCENT, AL 35178",Vincent,33.399231,-86.404985,30.20,NATIONAL PRIORITIES LIST (NPL) SITE LISTING NA...,Alabama,Shelby County,30102,...,0.00,0.00,0.00,30.23,14.50,9.02,16.00,84.00,7.02,30.49
2,3,"3017 BIRMINGHAM HWY, MONTGOMERY, AL 36108",Montgomery,32.367036,-86.372935,44.46,NPL SITE LISTING NARRATIVE\n\n\nNPL Site Narra...,Alabama,Montgomery County,6000,...,0.00,0.00,0.11,50.87,33.92,24.11,33.11,66.89,0.00,81.57
3,4,"12504 AND 12512 E. WHITTIER BOULEVARD, WHITTIE...",Whittier,33.969447,-118.043443,30.94,NPL SITE LISTING NARRATIVE\n\n\nNPL Site Narra...,California,Los Angeles County,502100,...,1.69,0.04,0.08,31.43,22.99,3.60,61.83,38.17,3.52,85.29
4,5,"ROUTE 950, FALLS CREEK, PA 15840",Falls Creek,41.145061,-78.804365,30.22,NATIONAL PRIORITIES LIST (NPL) SITE LISTING NA...,Pennsylvania,Jefferson County,950300,...,0.20,0.00,0.00,7.70,10.70,7.53,27.91,72.09,2.06,1.31


## Export the Data

### Pull the Data

In [4]:
rds_connection_string = "postgres:postgres@localhost:5432/project_2_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [5]:
superfund_sites_for_viz = pd.read_sql_query('select * from superfund_analysis_table', con=engine)

In [6]:
#### Drop the site_text column
superfund_sites_for_viz = superfund_sites_for_viz.drop(['site_text'], axis=1)

### Export the cleaned dataframe to csv

In [29]:
superfund_sites_for_viz.to_csv(viz_inspection_file, index = False)

### Export in GEOJSON format
(Much thanks to Geoff Boeing for his post "Exporting Python Data to GeoJSON":
https://geoffboeing.com/2015/10/exporting-python-data-geojson/)

In [30]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [31]:
cols = ['tract','address', 'site_score', 'med_hhd_inc_bg_acs_09_13', 'pct_no_health_ins_acs_09_13', 'pct_poc']
geojson = df_to_geojson(superfund_sites_for_viz, cols)

In [32]:
geojson

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'tract': 11300,
    'address': 'STATE HWY 235, CHILDERSBURG, AL 35044',
    'site_score': 36.83,
    'med_hhd_inc_bg_acs_09_13': 36534.0,
    'pct_no_health_ins_acs_09_13': 5.32,
    'pct_poc': 20.29},
   'geometry': {'type': 'Point', 'coordinates': [-86.3588071, 33.286432]}},
  {'type': 'Feature',
   'properties': {'tract': 30102,
    'address': '570 HIGHWAY 231 NORTH, VINCENT, AL 35178',
    'site_score': 30.2,
    'med_hhd_inc_bg_acs_09_13': 50219.0,
    'pct_no_health_ins_acs_09_13': 14.5,
    'pct_poc': 30.49},
   'geometry': {'type': 'Point', 'coordinates': [-86.4049851, 33.3992313]}},
  {'type': 'Feature',
   'properties': {'tract': 6000,
    'address': '3017 BIRMINGHAM HWY, MONTGOMERY, AL 36108',
    'site_score': 44.46,
    'med_hhd_inc_bg_acs_09_13': 30930.0,
    'pct_no_health_ins_acs_09_13': 33.92,
    'pct_poc': 81.57000000000001},
   'geometry': {'type': 'Point', 'coordinates': [-86.3729352,

In [33]:
output_filename = geojson_output_file
with open(output_filename, 'w') as output_file:
    # optional line for defining a variable here
    # output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 

### Export as a flat JSON

In [16]:
superfund_sites_for_viz.to_json(r'../03_Dataset/superfund_site_json.json', orient='records')

In [36]:
superfund_sites_for_viz.columns.values

array(['id', 'address', 'city', 'latitude', 'longitude', 'site_score',
       'state_name', 'county_name', 'tract', 'block_group',
       'tot_population_cen_2010', 'hispanic_cen_2010',
       'nh_blk_alone_cen_2010', 'nh_aian_alone_cen_2010',
       'nh_asian_alone_cen_2010', 'nh_nhopi_alone_cen_2010',
       'nh_sor_alone_cen_2010', 'college_acs_09_13',
       'no_health_ins_acs_09_13', 'med_hhd_inc_bg_acs_09_13',
       'aggregate_hh_inc_acs_09_13', 'tot_vacant_units_cen_2010',
       'renter_occp_hu_cen_2010', 'owner_occp_hu_cen_2010',
       'no_plumb_acs_09_13', 'med_house_value_bg_acs_09_13',
       'pct_hispanic_cen_2010', 'pct_nh_blk_alone_cen_2010',
       'pct_nh_aian_alone_cen_2010', 'pct_nh_asian_alone_cen_2010',
       'pct_nh_nhopi_alone_cen_2010', 'pct_nh_sor_alone_cen_2010',
       'pct_not_hs_grad_acs_09_13', 'pct_no_health_ins_acs_09_13',
       'pct_vacant_units_cen_2010', 'pct_renter_occp_hu_cen_2010',
       'pct_owner_occp_hu_cen_2010', 'pct_no_plumb_acs_09_13', 